In [1]:
news20 = spark.read.parquet("dbfs:/databricks-datasets/news20.binary/data-001/training")

In [2]:
news20.show()

In [3]:
news20.groupBy('topic').count().show()

In [4]:
news20.groupBy('label').count().show()

In [5]:
from pyspark.sql.functions import length

# Add a new column 'text_length' that contains the length of data in column 'text'
news20_with_lengths = news20.withColumn('text_length', length(news20['text']))

In [6]:
# Find average text in each topic
news20_with_lengths.groupBy('topic').avg('text_length').orderBy('avg(text_length)').show(truncate = False)

In [7]:
a